In [136]:
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter, ParameterVector
from qiskit.quantum_info import SparsePauliOp, Pauli
import networkx as nx
import numpy as np
from scipy.optimize import minimize
from qiskit.primitives import Estimator

def custom_ansatz(num_qubits):
    qc = QuantumCircuit(num_qubits)
    
    # Define a parameter vector for the circuit
    theta = ParameterVector('θ', num_qubits)
    
    # Apply parameterized rotations around the Y-axis for each qubit
    for i in range(num_qubits):
        qc.ry(theta[i], i)
    
    # Apply a chain of CNOT gates to entangle adjacent qubits
    for i in range(num_qubits - 1):
        qc.cx(i, i + 1)
    
    # Apply a CNOT between the first and the last qubit
    qc.cx(num_qubits - 1, 0)
    
    return qc, theta

def maxcut_hamiltonian(graph, num_qubits):
    """Generate the MaxCut Hamiltonian for a given graph."""
    paulis = []
    coeffs = []
    
    # Generate terms for the Hamiltonian
    for u, v in graph.edges:
        # Add -1/2 * Z_u Z_v term
        pauli_str = ['I'] * num_qubits
        pauli_str[u] = 'Z'
        pauli_str[v] = 'Z'
        paulis.append(Pauli("".join(pauli_str)))
        coeffs.append(0.5)
        
        #Add +1/2 term (identity operator)
        paulis.append(Pauli("I" * num_qubits))
        coeffs.append(-0.5)
    
    return SparsePauliOp(paulis, coeffs)

# Updated cost function to use custom ansatz
def cost_function(params, ansatz, hamiltonian, num_qubits):
    qc, theta = ansatz(num_qubits)
    
    # Create a dictionary to assign parameters to the circuit
    param_dict = {theta[i]: params[i] for i in range(len(theta))}
    
    # Assign parameters to the circuit
    bound_circuit = qc.assign_parameters(param_dict)
    
    # Create an estimator primitive
    estimator = Estimator()
    
    # Calculate the expectation value of the Hamiltonian
    result = estimator.run(bound_circuit, [hamiltonian])
    expectation_value = result.result().values[0]
    
    return expectation_value

# Updated optimization loop to use custom ansatz
def optimization_loop(hamiltonian, num_qubits, optimizer, initial_params):
    def cost(params):
        return cost_function(params, custom_ansatz, hamiltonian, num_qubits)
    
    result = minimize(cost, initial_params, method=optimizer, options={'maxiter': 1000, 'tol': 1e-6})
    
    return result.fun, result.x

# Define the number of qubits
num_qubits = 12  # Example number of qubits

# Define the graph for MaxCut
graph = nx.complete_graph(num_qubits)  # All-to-all graph

# Generate the MaxCut Hamiltonian
hamiltonian = maxcut_hamiltonian(graph, num_qubits)

# Generate initial parameters for the custom ansatz (one parameter per qubit)
initialization_params = np.random.uniform(0, 2 * np.pi, num_qubits)
#initialization_params = np.zeros(num_qubits)
# Define the optimizer
optimizer = 'COBYLA'

# Perform the optimization manually using the custom ansatz
optimal_value, optimal_params = optimization_loop(hamiltonian, num_qubits, optimizer, initialization_params)
print(f'Optimal energy: {optimal_value}')
#print(f'Optimal parameters: {optimal_params}')


Optimal energy: -35.99999999999982


In [115]:
import networkx as nx
import numpy as np
from qiskit.quantum_info import Pauli, SparsePauliOp

def maxcut_hamiltonian(graph, num_qubits):
    """Generate the MaxCut Hamiltonian for a given graph."""
    paulis = []
    coeffs = []
    
    # Generate terms for the Hamiltonian
    for u, v in graph.edges:
        # Add -1/2 * Z_u Z_v term
        pauli_str = ['I'] * num_qubits
        pauli_str[u] = 'Z'
        pauli_str[v] = 'Z'
        paulis.append(Pauli("".join(pauli_str)))
        coeffs.append(0.5)
        
        #Add +1/2 term (identity operator)
        paulis.append(Pauli("I" * num_qubits))
        coeffs.append(-0.5)
    
    # Construct SparsePauliOp
    H = SparsePauliOp(paulis, coeffs)
    return H

num_qubits = 4
graph = nx.complete_graph(num_qubits)


# Generate the MaxCut Hamiltonian
H = maxcut_hamiltonian(graph, num_qubits)

# Convert SparsePauliOp to matrix and compute eigenvalues
H_matrix = H.to_matrix()
eigenvalues = np.linalg.eigvalsh(H_matrix)
min_eigenvalue = np.min(eigenvalues)

# Display results
#print("Hamiltonian matrix:\n", H_matrix)
print("\nEigenvalues:", eigenvalues)
print("\Maximum eigenvalue:", min_eigenvalue)



Eigenvalues: [-4. -4. -4. -4. -4. -4. -3. -3. -3. -3. -3. -3. -3. -3.  0.  0.]
\Maximum eigenvalue: -4.0


<>:42: SyntaxWarning: invalid escape sequence '\M'
<>:42: SyntaxWarning: invalid escape sequence '\M'
/var/folders/hj/r8f4lnjd3z33vqn7mbr8635h2711rk/T/ipykernel_96755/556306526.py:42: SyntaxWarning: invalid escape sequence '\M'
  print("\Maximum eigenvalue:", min_eigenvalue)
